In [1]:
import os
from dotenv import load_dotenv 
import json
from utils import *
from web3 import Web3
import pandas as pd
from pprint import pprint

In [2]:
# Get the config file
configObj = ConfigManager("config.json")
appInfo, configData = configObj.load_config()
nodeUrl = appInfo["alchemy_url"]+appInfo["alchemy_key"]
# configObj.save_config(config=configData)

In [9]:
# Connect to Alchemy's Ethereum node
web3 = Web3(Web3.HTTPProvider(nodeUrl))

# Check if connection is successful
if web3.is_connected():
    # Get the latest block
    latest_block = web3.eth.get_block('latest')
    
    print(f"Latest block: {latest_block['number']}")
else:
    raise Exception("Failed to connect to the Ethereum node. Are you sure you have entered the API key correctly?")

print(f"{latest_block['number'] - configData['latest_block_checked']} blocks to check")

Latest block: 21535996
12856 blocks to check


In [10]:
# Standard ERC20 ABI (minimum required functions)
ERC20_ABI = [
    # Basic token info
    {"inputs": [], "name": "name", "outputs": [{"type": "string"}], "stateMutability": "view", "type": "function"},
    {"inputs": [], "name": "symbol", "outputs": [{"type": "string"}], "stateMutability": "view", "type": "function"},
    {"inputs": [], "name": "decimals", "outputs": [{"type": "uint8"}], "stateMutability": "view", "type": "function"},
    {"inputs": [], "name": "totalSupply", "outputs": [{"type": "uint256"}], "stateMutability": "view", "type": "function"},
    
    # Balance and allowance
    {"inputs": [{"type": "address"}], "name": "balanceOf", "outputs": [{"type": "uint256"}], "stateMutability": "view", "type": "function"},
    {"inputs": [{"type": "address"}, {"type": "address"}], "name": "allowance", "outputs": [{"type": "uint256"}], "stateMutability": "view", "type": "function"},
    
    # Transfer events
    {"anonymous": False, "inputs": [{"indexed": True, "type": "address"}, {"indexed": True, "type": "address"}, {"indexed": False, "type": "uint256"}], "name": "Transfer", "type": "event"},
    {"anonymous": False, "inputs": [{"indexed": True, "type": "address"}, {"indexed": True, "type": "address"}, {"indexed": False, "type": "uint256"}], "name": "Approval", "type": "event"}
]

def get_token_details(token_address, node_url):
    # Connect to Ethereum node
    web3 = Web3(Web3.HTTPProvider(node_url))
    
    # Convert to checksum address
    token_address = web3.to_checksum_address(token_address)
    
    # Validate address
    if not web3.is_address(token_address):
        raise Exception("Invalid address")
    
    if  len(web3.eth.get_code(token_address)) != 0:
        try:
            # Create contract instance
            contract = web3.eth.contract(address=token_address, abi=ERC20_ABI)
            
            # Get basic token information
            token_info = {
                'address': token_address,
                'name': contract.functions.name().call(),
                'symbol': contract.functions.symbol().call(),
                'decimals': contract.functions.decimals().call(),
                'total_supply': contract.functions.totalSupply().call()
            }
            
            # Get contract code size (to verify it's a contract)
            token_info['is_contract'] = "True"
            
            
            return token_info
        except:
            token_info = {
                'address': token_address,
                'name': "-",
                'symbol': "-",
                'decimals': "-",
                'total_supply': "-"
            }
            token_info['is_contract'] = "True"
            return token_info
    else:
        # Not a conteract
        return None

newTokens = pd.DataFrame(columns=['address', 'name', 'symbol', 'decimals', 'total_supply', 'is_contract'])

# Go through past blocks to find conteract creation events
for i in range(configData['latest_block_checked'], latest_block['number']):
    print(f"===== Checking block {i} ======")
    block = web3.eth.get_block(i, True)
    for tx in block.transactions:
        if tx["to"] == None:
            tx_receipt = web3.eth.get_transaction_receipt(tx['hash'])
            contract_address = tx_receipt['contractAddress']
            contract_code = web3.eth.get_code(contract_address)
            if contract_code != '0x':
                print(f"Contract created at address {contract_address} in block {i}")
                __details = get_token_details(contract_address, nodeUrl)
                newTokens = pd.concat([newTokens, pd.DataFrame({'address': __details["address"],
                            'name': __details["name"],
                            'symbol': __details["symbol"],
                            'decimals': __details["decimals"],
                            'total_supply': __details["total_supply"],
                            'is_contract': __details["is_contract"]}, index=[0])], ignore_index=True)

===== Checking block 21523140 ======
Contract created at address 0x375dd46eB094e1939D84709fCA47583d3fd44c20 in block 21523140
===== Checking block 21523141 ======
Contract created at address 0xCed6b74bB9DB1432120b4663c013b64EC3ebb115 in block 21523141
===== Checking block 21523142 ======
===== Checking block 21523143 ======
===== Checking block 21523144 ======
===== Checking block 21523145 ======
===== Checking block 21523146 ======
===== Checking block 21523147 ======
Contract created at address 0x5D2213e64BDb909EFEE29fd8333D200700D52FEb in block 21523147
===== Checking block 21523148 ======
Contract created at address 0xDC59C60Cdceb9f648AaC886069BEE1Bd48c2a553 in block 21523148
===== Checking block 21523149 ======
===== Checking block 21523150 ======
Contract created at address 0x83E4501bc027067815738beeCB2BFd6B2E643a94 in block 21523150
===== Checking block 21523151 ======
===== Checking block 21523152 ======
===== Checking block 21523153 ======
Contract created at address 0x92c4574

KeyboardInterrupt: 

In [5]:
block = web3.eth.get_block(21521270, full_transactions=True)

if block:
    print(f"Block Number: {block.number}")
    print(f"Block Hash: {block.hash.hex()}")
    print(f"Timestamp: {block.timestamp}")
    print(f"Number of transactions: {len(block.transactions)}")

Block Number: 21521270
Block Hash: 15ec45fd76ea42d5e845ebf90e54327b6fbc13b818380ca1c84701ea0fb2f0e0
Timestamp: 1735633775
Number of transactions: 148


In [11]:
newTokens.to_csv('new_tokens.csv', index=False)

In [54]:
a = get_token_details("0xf9ca9523e5b5a42c3018c62b084db8543478c400", nodeUrl)
pprint(a)

{'address': '0xF9Ca9523E5b5A42C3018C62B084Db8543478C400',
 'decimals': 18,
 'is_contract': True,
 'name': 'Data Lake Token',
 'symbol': 'LAKE',
 'total_supply': 7500000000000000000000000000}


In [51]:
token_address = web3.to_checksum_address("0xf9ca9523e5b5a42c3018c62b084db8543478c400")
contract = web3.eth.contract(address=token_address, abi=ERC20_ABI)
contract.functions.name().call()

'Data Lake Token'

In [7]:
import psycopg2
from utils import dbUtils


# Connect to the database

db = dbUtils(user = "postgres", password = "1234", host  = "localhost", port =  "5432")

# See if database exists
if db.database_exists("screenerDB"):
    if not db.table_exists("screenerDB", "tokens"):
        print("Creating table 'tokens' in the database 'screenerDB'")
        # Cursor
        db._connect_to_db("screenerDB")
        con = db._conn
        cur = con.cursor()
        
        # Make a table for the new tokens
        cur.execute("""
            CREATE TABLE IF NOT EXISTS tokens (
                id SERIAL PRIMARY KEY,
                address VARCHAR(255),
                name VARCHAR(255),
                symbol VARCHAR(255),
                chain_name VARCHAR(255),
                decimals INT,
                inception_time BIGINT,
                inception_block BIGINT,
                total_supply VARCHAR(255)
            )
        """)
        
        con.commit()
        
        # close the connection
        con.close()
    else:
        pass
else:
    print(f"Database doesn't exist. First create the database with the name 'screenerDB'")





Creating table 'tokens' in the database 'screenerDB'


In [3]:
from datetime import datetime
from utils import dbUtils
db = dbUtils(user = "postgres", password = "1234", host  = "localhost", port =  "5432")
data = {
    'address': "0x248A791B9b3E0e17641A5D0E306B8485403432a9",
    'name': "PopKitty",
    'symbol': "POPKI",
    'chain_name': "Ethereum",
    'decimals': 9,
    'inception_time': datetime.now().timestamp(),
    'inception_block': 2155211421,
    'total_supply': str(100000000000000000000000000)
}
db.insert_row("screenerDB", "tokens", data)

(True, 24)

In [6]:
success, rows = db.get_rows(appInfo["database_name"], "tokens", dataframe=True)
rows

,address,name,symbol,chain_name,decimals,inception_time,inception_block,total_supply
id,,,,,,,,
1,0x111,PopKitty,POPKI,Ethereum,9,1735827041,2155211421,100000000000000000000000000
2,0x248A791B9b3E0e17641A5D0E306B8485403432a9,PopKitty,POPKI,Ethereum,9,1735825570,2155211421,100000000000000000000000000
4,0x248A791B9b3E0e17641A5D0E306B8485403432a9,PopKitty,POPKI,Ethereum,9,1735825570,2155211421,100000000000000000000000000
5,0x248A791B9b3E0e17641A5D0E306B8485403432a9,PopKitty,POPKI,Ethereum,9,1735825571,2155211421,100000000000000000000000000
6,0x248A791B9b3E0e17641A5D0E306B8485403432a9,PopKitty,POPKI,Ethereum,9,1735825571,2155211421,100000000000000000000000000
7,0x248A791B9b3E0e17641A5D0E306B8485403432a9,PopKitty,POPKI,Ethereum,9,1735825571,2155211421,100000000000000000000000000
8,0x248A791B9b3E0e17641A5D0E306B8485403432a9,PopKitty,POPKI,Ethereum,9,1735825571,2155211421,100000000000000000000000000
9,0x248A791B9b3E0e17641A5D0E306B8485403432a9,PopKitty,POPKI,Ethereum,9,1735825571,2155211421,100000000000000000000000000
10,0x248A791B9b3E0e17641A5D0E306B8485403432a9,PopKitty,POPKI,Ethereum,9,1735825572,2155211421,100000000000000000000000000


In [5]:
success, count = db.delete_row(appInfo["database_name"], "tokens", "id = %s", 3 )

In [4]:
data["address"] = "0x111"
db.update_row(
    appInfo["database_name"],
    "tokens",
    data,
    "id = %s",
    1
)

(True, 1)

In [5]:
data = {
    'address': "0x248A791B9b3E0e17641A5D0E306B8485403432a9",
    'name': "PopKitty",
    'symbol': "POPKI",
    'chain_name': "Ethereum",
    'decimals': 9,
    'inception_time': datetime.now().timestamp(),
    'inception_block': 2155211421,
    'total_supply': str(100000000000000000000000000)
}
db.insert_row(appInfo["database_name"], "tokens", data)

(True, 21)